In [ ]:
import time
import copy
import random
import numpy as np
import itertools


import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import datetime
import string

home_folder = './figures'

from tqdm import tqdm
import mapel.elections as mapel 

In [ ]:
# Helper functions
rcParams.update({
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
        'figure.figsize': (10,6),
})

def file_str():
    """ Auto-generates file name."""
    now = datetime.datetime.now()
    return now.strftime("H%HM%MS%S_%m-%d-%y")

rand_string = lambda length: ''.join(random.choice(string.ascii_lowercase) for i in range(length))

def pdf_savefig():
    """ Saves figures as pdf """
    fname = file_str()+rand_string(5)
    plt.savefig(home_folder+f"/{fname}.pdf", bbox_inches="tight")

def eps_savefig():
    """ Saves figure as encapsulated postscript file (vector format)
        so that it isn't pixelated when we put it into a pdf. """
    pdf_savefig()

    


### Voting rules #######

In [ ]:
def get_borda_score(election) -> list:
    """ Return: List with all Borda scores """
    scores = [0 for _ in range(election.num_candidates)]
    for vote in election.votes:
        for c in range(election.num_candidates):
            scores[vote[c]] += c
    return np.array(scores)

def get_sntv_score(election) -> list:
    """ Return: List with all SNTV scores """
    sntv = [_ == 0 for _ in range(election.num_candidates)]
    scores = [0 for _ in range(election.num_candidates)]
    for vote in election.votes:
        for c in range(1):
            scores[vote[c]] += sntv[c]
    return np.array(scores)

def get_bloc_score(election, b) -> list:
    """ Return: List with all SNTV scores """
    bloc = [_ < b for _ in range(election.num_candidates)]
    scores = [0 for _ in range(election.num_candidates)]
    for vote in election.votes:
        for c in range(b):
            scores[vote[c]] += bloc[c]
    return np.array(scores)




### Generative model $\hat{\mu}$ given $\succ\hspace{2mm} \sim \mu$

In [ ]:
# Bias function (input: election, phi, groups, output: election_hat)
def get_swapping_bias(election, phi, g1, g2, t_frac):
    
    assert phi < 1
    
    # biased election
    election_hat = copy.deepcopy(election)
    
    # to speed up containment check 
    set_g1 = set(g1)
    set_g2 = set(g2) 
    
    for vote in election_hat.votes: 
        cnt = 0
        maximum_swaps = 0

        # compute maximum number of swaps votes
        for i in np.arange(election_hat.num_candidates):
            if vote[i] in set_g1:
                maximum_swaps += cnt
            if vote[i] in set_g2:
                cnt += 1
            
        t = int(t_frac * maximum_swaps)
        
        # Compute positions 
        pos = [0 for i in range(len(vote))]
        for i in range(len(vote)):
            pos[vote[i]] = i
        
        # Vectorize the computation of V
        p1 = np.array([pos[i] for i in g1])
        p2 = np.array([pos[i] for i in g2])
        v1 = phi ** p1
        v2 = phi ** (-p2)
        
        V = np.outer(v1, v2)
        V = np.where(V > 1, 0, V)
        
        for ind in range(t):
            # Break if no more swaps possible 
            if np.sum(V) == 0: break

            # Sample the pair swapped 
            i = random.choices(np.arange(V.shape[0]), k = 1, weights = np.sum(V, axis=1) / np.sum(V))[0] 
            j = random.choices(np.arange(V.shape[1]), k = 1, weights = V[i, :] / np.sum(V[i, :]))[0] 
             
            # Only do O(n+m) updates to W each step
            V[i, :] = np.where(V[i, :] > 1, 0, V[i, :])
            V[:, j] = np.where(V[:, j] > 1, 0, V[:, j])
            p1[i], p2[j] = p2[j], p1[i]  

        # Generate biased votes using p1 and p2
        for j in range(len(vote)):
            vote[j] = -1
        
        for ijk, i in enumerate(g1):
            vote[p1[ijk]] = i
         
        for ijk, i in enumerate(g2): 
            vote[p2[ijk]] = i
             
    return election_hat

### Metrics functions #######

In [ ]:

def get_top_k(scores, k) -> list:
    #print(f'get_top_k, k={k}, len(scores)={len(scores)}')
    assert len(scores) >= k
    return np.argsort(-scores)[:k]

def get_s_ell(scores, k, l, g1, g2) -> list:
    scores_g1 = scores[g1]
    scores_g2 = scores[g2]
    
    cand_g1 = np.argsort(-scores_g1)[:k-l]
    cand_g2 = np.argsort(-scores_g2)[:l] 
    
    assert len(g1) >= k-l
    assert len(g2) >= l
    
    cand_g1 = list(g1[cand_g1])
    cand_g2 = list(g2[cand_g2])
    
    return np.array(cand_g1 + cand_g2)


In [ ]:
# Select committee maximizing score and
# output the properties of the selected committee
def get_committee_score_const(scores_hat, g1, g2, l, k, scores):
    if l != -1:
        s_ell = get_s_ell(scores_hat, k, l, g1, g2)
    else:
        s_ell = get_top_k(scores_hat, k) 
        
    assert len(s_ell) == k
    return np.sum(scores[s_ell])

def get_committee_score_opt(scores_hat, g1, g2, l, k, scores):
    s = get_top_k(scores, k)
    
    assert len(s) == k
    return np.sum(scores[s])

In [ ]:
for i in tqdm(range(50)):
    n = 40
    m = 40
    k = 20
    g1 = np.arange(m//2)
    g2 = np.arange(m//2) + m//2
    election = mapel.generate_election(culture_id = 'impartial_culture', num_candidates=m, num_voters=n)
    election_hat = get_swapping_bias(election, 0.01, g1, g2, 0.9)
    scores = get_borda_score(election)
    scores_hat = get_borda_score(election_hat)
    
    
    
    sc = np.sort(-scores)[:k]
    assert len(sc) == k

    # find the best committees with/without constraint
    score_no_const = get_committee_score_const(scores_hat, g1, g2, -1, k, scores)
    score_ye_const = get_committee_score_const(scores_hat, g1, g2, k//2, k, scores)
    score_opt = get_committee_score_opt(scores, g1, g2, 0, k, scores)
    
    if score_opt >= score_no_const and  score_opt >= score_ye_const:
        continue
    else:
        print(scores)
        print(sorted(scores))
        print(f'np.sum(sc)\t=\t{-np.sum(sc)}')
        print(f'score_opt\t=\t{score_opt}')
        print(f'score_ye_const\t=\t{score_ye_const}')
        print(f'score_no_const\t=\t{score_no_const}')
        print()        
        break


### Parameters 

In [ ]:
p = 2 
k = 10
ITER = 40
 

get_election = lambda x, y: mapel.generate_election(culture_id = 'norm-mallows',\
                                                    num_candidates = x,\
                                                    num_voters = y)

rules = ['Borda', '10-Bloc', 'SNTV']

colors = {}
colors['Borda'] = '#377eb8'
colors['10-Bloc'] = '#ff7f00'
colors['SNTV'] = '#4daf4a'

get_score = {}
get_score['Borda'] = lambda x: get_borda_score(x)
get_score['SNTV'] = lambda x: get_sntv_score(x)
get_score['10-Bloc'] = lambda x: get_bloc_score(x, 10)

### Run simulation

In [ ]:
for n, m in itertools.product([25, 50, 100], [50]): 
    for dist in ['impartial_culture', 'urn', 'mallows', 'norm-mallows', 'conitzer']:
        get_election = lambda x, y: mapel.generate_election(culture_id = dist,\
                                                    num_candidates = x,\
                                                    num_voters = y)
        lb_list = np.round(np.linspace(0, 1, 10) * min(k, m/2)) 

        frac_swaps_list = np.linspace(0, 1, 8)

        g1 = np.random.choice(np.arange(m), m//2, replace=False)
        set_g1 = set(g1)
        g2 = []
        for i in np.arange(m):
            if i not in set_g1:
                g2.append(i)
        g2 = np.array(g2)
        
        best_ratio_ye_const = {r: [] for r in rules} 
        best_ratio_ye_const_std = {r: [] for r in rules} 

        phi = 0.5

        for frac_swaps in tqdm(frac_swaps_list):
            for r in rules: # Append best ratio
                best_ratio_ye_const[r].append(-1)
                best_ratio_ye_const_std[r].append(-1)

            for lb in np.arange(0, k+1, 2):
                lb = int(lb) 

                ratio_no_const = {r: [] for r in rules} 
                ratio_ye_const = {r: [] for r in rules} 

                for ijk in range(ITER):  
                    election = get_election(m, n)
                    election_hat = get_swapping_bias(election, phi, g1, g2, frac_swaps)

                    for r in rules:
                        scores = get_score[r](election) 
                        scores_hat = get_score[r](election_hat)  

                        # find the committees: with constraints and the optimal committee 
                        score_ye_const = get_committee_score_const(scores_hat, g1, g2, lb, k, scores)
                        score_opt = get_committee_score_opt(scores, g1, g2, 0, k, scores)
                        
                        #print(f'scores={scores}')
                        #print(f'score_opt={score_opt}\tscore_ye_const={score_ye_const}')

                        assert(score_opt >= score_ye_const)  
                    
                        # evaluations 
                        ratio_ye_const[r].append(score_ye_const / (score_opt + 1e-18))   

                ratio_ye_const_mean = {r: [] for r in rules} 
                ratio_ye_const_std = {r: [] for r in rules} 
                for r in rules: # Updates to ratio  
                    ratio_ye_const_mean[r] = np.mean(ratio_ye_const[r]) 
                    ratio_ye_const_std[r] = np.std(ratio_ye_const[r]) / np.sqrt(ITER) 

                    assert(ratio_ye_const_mean[r] <= 1 + 1e-5)

                for r in rules: # Updates to best ratio 
                    if best_ratio_ye_const[r][-1] < ratio_ye_const_mean[r]:
                        best_ratio_ye_const_std[r][-1] = ratio_ye_const_std[r]

                    best_ratio_ye_const[r][-1] = max(best_ratio_ye_const[r][-1], ratio_ye_const_mean[r]) 

        ####################################
        # PLOT 1
        ####################################
        plt.title(f'Dist={dist}, n={n}, m={m}, k={k}, Bias-model=Swapping-Bias, ITER={ITER}')
        plt.ylabel('${\\rm OPT}^{-1} \cdot \max_\ell\ F(S_\ell)$', fontsize=26)
        plt.xlabel('Fraction of swaps $\\lambda$', fontsize=30)
        plt.ylim(0.6, 1.01)

        for r in rules:
            plt.errorbar(frac_swaps_list, best_ratio_ye_const[r], yerr=best_ratio_ye_const_std[r], linewidth = 8, label=r, color=colors[r])

        plt.legend(fontsize=30, shadow=False)
        plt.tick_params(axis='both', which='major', labelsize=30)
        plt.show()
        #pdf_savefig()
        plt.close()
         
        ####################################
        # PLOT 2
        ####################################
        plt.title(f'Dist={dist}, n={n}, m={m}, k={k}, Bias-model=Swapping-Bias, ITER={ITER}')
        plt.ylabel('${\\rm OPT}^{-1} \cdot \max_\ell\ F(S_\ell)$', fontsize=26)
        plt.xlabel('Fraction of swaps $\\lambda$', fontsize=30)
        plt.ylim(0.6, 1.01)
        
        for r in ['Borda', 'SNTV']:
            plt.errorbar(frac_swaps_list, best_ratio_ye_const[r], yerr=best_ratio_ye_const_std[r], linewidth = 8, label=r, color=colors[r])

        plt.legend(fontsize=26, shadow=False)
        plt.tick_params(axis='both', which='major', labelsize=30)
        plt.show()
        #pdf_savefig()
        plt.close()
